In [222]:
docs = []
query = ""

In [223]:
import string
import math

In [224]:
for i in range(1, 7):
    f = open(f'/home/jass/repos/sem8/20XW86-IR-LAB/bim/testdocs/d{i}.txt', 'r')
    docs.append(f.read())

In [225]:
docs

['information requirement: query considers the user feedback as information\nrequirement to search.',
 'information retrieval: query depends on the model of information retrieval\nused.',
 'prediction problem: Many problems in information retrieval can be viewed as\nprediction problems',
 'search: A search engine is one of applications of information retrieval models.',
 'Feedback: feedback is typically used by the system to modify the query and\n\nimprove prediction',
 'information retrieval: ranking in information retrieval algorithms depends on\n\nuser query']

In [226]:
def transform_text(text):
    text = text.split()
    y = []
    for word in text:
        char_list = []
        for c in word:
            if c not in string.punctuation:
                char_list.append(c)
        
        lcase_word  = ''.join(char_list).lower()

        if lcase_word.isalnum():
            y.append(lcase_word)

    return ' '.join(y) 

    
        

In [227]:
processed_docs = []

In [228]:
for doc in docs:
    processed_docs.append(transform_text(doc))

In [229]:
processed_docs

['information requirement query considers the user feedback as information requirement to search',
 'information retrieval query depends on the model of information retrieval used',
 'prediction problem many problems in information retrieval can be viewed as prediction problems',
 'search a search engine is one of applications of information retrieval models',
 'feedback feedback is typically used by the system to modify the query and improve prediction',
 'information retrieval ranking in information retrieval algorithms depends on user query']

In [230]:
TD_matrix = []

corpus_sets = []
master_set = set()
for doc in processed_docs:
    hs = set()
    for word in doc.split():
        hs.add(word)
        
        if word not in master_set:
            master_set.add(word)
    corpus_sets.append(hs)

master_set = sorted(list(master_set))

In [231]:
master_set[:16]

['a',
 'algorithms',
 'and',
 'applications',
 'as',
 'be',
 'by',
 'can',
 'considers',
 'depends',
 'engine',
 'feedback',
 'improve',
 'in',
 'information',
 'is']

In [232]:
for i in range(6):
    vector = [0]*len(master_set)

    for idx,word in enumerate(master_set):
        if word in corpus_sets[i]:
            vector[idx] = 1

    TD_matrix.append(vector)

In [233]:
print(TD_matrix[:10])
print(corpus_sets)

[[0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0], [0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1], [1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0]]
[{'feedback', 'requirement', 'considers', 'user', 'query', 'as', 'to', 'the', 'information', 'search'}, {'on', 'model', 'of', 'used', 'query', 'depends', 'retrieval', 'the', 'information'}, {'in', 'prediction', 'problem', 'problems', 'many', 'retrieval', 'viewed', 'as', 'information', 'be', 'can'}, {'m

In [234]:
def formulate_vector(query):
    q_hs = set()
    
    for word in query.split():
        q_hs.add(word)

    vector = [0]*len(master_set)

    for idx,word in enumerate(master_set):
        if word in q_hs:
            vector[idx] = 1
    
    return vector

In [235]:
def DF(word, corpus_sets):
    count = 0

    for hs in corpus_sets:
        if word in hs:
            count += 1
    
    return count

def TRk(Pk, Qk):
    return math.log(Pk*(1-Qk)/Qk*(1-Pk))

def RK(word, corpus_sets, rdoc_ids):
    count = 0
    rset_ids = set(rdoc_ids)

    for i in range(len(corpus_sets)):
        if i in rset_ids and word in corpus_sets[i]:
                count += 1

    return count  

In [236]:
def phase1(query, TRk_phase1, corpus_sets, query_vector, TD_matrix, master_set):
    print("Phase 1 begins")
    for word in master_set:
        Pk = 0.5
        Qk = (DF(word, corpus_sets)+0.5)/(len(corpus_sets)+1)
        print(Pk, Qk)
        trk = TRk(Pk, Qk)

        TRk_phase1[word] = trk
        
    print("TRk phase 1", TRk_phase1)

    for dv in TD_matrix:
        for i,word in enumerate(master_set):
            if dv[i]==1:
                dv[i] *= TRk_phase1[word]

    dot_prod = []

    for i,dv in enumerate(TD_matrix):
        dot_prod.append([i, np.dot(np.array(dv), np.array(query_vector))])    

    dot_prod.sort(key= lambda x:x[1], reverse=True)
    print("Phase1: ", dot_prod)
    return [x for x,y in dot_prod[:3]]

In [237]:
def phase2(query, TRk_phase2, corpus_sets, query_vector, TD_matrix, master_set, rdoc_ids):
    print("Phase 2 begins ..")
    Nr = len(rdoc_ids)

    for word in master_set:
        Rk = RK(word, corpus_sets, rdoc_ids)
        Pk = (Rk+0.5)/(Nr+1)
        Qk = (DF(word, corpus_sets)-Rk+0.5)/(len(corpus_sets)-Nr+1)
        print(Pk, Qk)
        trk = TRk(Pk, Qk)

        TRk_phase2[word] = trk
    print("TRk phase 2", TRk_phase2)
    for dv in TD_matrix:
        for i,word in enumerate(master_set):
            if dv[i]==1:
                dv[i] *= TRk_phase2[word]

    dot_prod = []

    for i,dv in enumerate(TD_matrix):
        dot_prod.append([i, np.dot(np.array(dv), np.array(query_vector))])    

    dot_prod.sort(key= lambda x:x[1], reverse=True)
    print("Phase 2:", dot_prod)
    return [x+1 for x,y in dot_prod]

In [238]:
def BIM(query, corpus_sets):
    query = transform_text(query)
    query_vector = formulate_vector(query)
    rel1 = phase1(query, {}, corpus_sets, query_vector, TD_matrix, master_set)
    rel2 = phase2(query, {}, corpus_sets, query_vector, TD_matrix, master_set, rel1)

    return rel2

In [239]:
query = "Importance of user feedback in retrieval?"
BIM(query, corpus_sets)

Phase 1 begins
0.5 0.21428571428571427
0.5 0.21428571428571427
0.5 0.21428571428571427
0.5 0.21428571428571427
0.5 0.35714285714285715
0.5 0.21428571428571427
0.5 0.21428571428571427
0.5 0.21428571428571427
0.5 0.21428571428571427
0.5 0.35714285714285715
0.5 0.21428571428571427
0.5 0.35714285714285715
0.5 0.21428571428571427
0.5 0.35714285714285715
0.5 0.7857142857142857
0.5 0.35714285714285715
0.5 0.21428571428571427
0.5 0.21428571428571427
0.5 0.21428571428571427
0.5 0.21428571428571427
0.5 0.35714285714285715
0.5 0.35714285714285715
0.5 0.21428571428571427
0.5 0.35714285714285715
0.5 0.21428571428571427
0.5 0.21428571428571427
0.5 0.6428571428571429
0.5 0.21428571428571427
0.5 0.21428571428571427
0.5 0.6428571428571429
0.5 0.35714285714285715
0.5 0.21428571428571427
0.5 0.5
0.5 0.35714285714285715
0.5 0.21428571428571427
0.5 0.35714285714285715
0.5 0.35714285714285715
0.5 0.21428571428571427
TRk phase 1 {'a': -0.08701137698962969, 'algorithms': -0.08701137698962969, 'and': -0.087011

[5, 1, 2, 3, 4, 6]